# Data Transformation (SQL)

## Assignment Details

In this project, we'll be working with one month of data from sensors in buildings at UC Berkeley. This is a very typical real-world dataset—i.e. it's kind of a mess. The full dataset contains a giant `data` table of many billions of sensor readings over the course of a decade; we will look at a single month of that data. It also contains a variety of other tables that contextualize the readings.

In [3]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

In [4]:
%reload_ext sql
%sql postgresql://jovyan@127.0.0.1:5432/postgres

## Loading Up the Database

In [5]:
import subprocess
call = subprocess.run(["psql", "-h", "localhost", \
                       "-tAc", "SELECT 1 FROM pg_database WHERE datname='ucb_buildings'", "template1"], \
                      stdout=subprocess.PIPE, text=True)
if call.stdout == "1\n":
    !psql postgresql://localhost:5432/ucb_buildings -c 'SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity WHERE datname = current_database()  AND pid <> pg_backend_pid();'

# then delete and recreate database
!psql -h localhost -c 'DROP DATABASE IF EXISTS ucb_buildings'
!psql -h localhost -c 'CREATE DATABASE ucb_buildings'
!gunzip -c data/proj3.sql.gz | psql -h localhost -d ucb_buildings -f -

 pg_terminate_backend 
----------------------
(0 rows)

DROP DATABASE
CREATE DATABASE
SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
CREATE EXTENSION
COMMENT
CREATE EXTENSION
COMMENT
CREATE FUNCTION
ALTER FUNCTION
CREATE AGGREGATE
ALTER AGGREGATE
SET
SET
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE MATERIALIZED VIEW
ALTER TABLE
CREATE TABLE
ALTER TABLE
COPY 92
COPY 102386
COPY 100000
COPY 381
COPY 9509
COPY 26183
COPY 5276
COPY 12
ALTER TABLE
ALTER TABLE
ALTER TABLE
REFRESH MATERIALIZED VIEW


In [6]:
%sql postgresql://jovyan@127.0.0.1:5432/ucb_buildings

Connecting and switching to connection postgresql://jovyan@127.0.0.1:5432/ucb_buildings

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


<!-- BEGIN QUESTION -->

## Question 1: Unboxing the Data


<!-- END QUESTION -->

<br><br>

---
### Question 1b

The diagram claims that `buildings_site_mapping` has a **many-to-many relationship** with `real_estate_metadata`. Let's validate that. 

Find the **distinct** values of `buildings_site_mapping.building` that match multiple tuples in `real_estate_metadata.building_name`, and for each such value of `buildings_site_mapping.building` return the matches as JSON via `json_agg(real_estate_metadata)`. Your output should contain the building and the `json_agg` in that order. **Order your final result by building ascending**.

**Hint:** You should use a CTE to find the distinct buildings of `buildings_site_mapping` before applying necessary table joins.

In [9]:
%%sql --save query_1b result_1b <<

with tbl as (
    select distinct building 
    from buildings_site_mapping
)
    
select b.building, json_agg(r)
from real_estate_metadata as r
JOIN tbl b
ON b.building = r.building_name
group by b.building
having count(*) > 1
order by b.building

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

5 rows affected.

In [10]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_1b = %sqlcmd snippets query_1b
grading_util.save_results("result_1b", query_1b, result_1b)
result_1b

building,json_agg
ALUMNI HOUSE,"[{'location': 'BERKELEY', 'building': '1215', 'building_name': 'ALUMNI HOUSE', 'address': 'CORE C22A2M0PUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 15590, 'bldg_asf': 8719, 'levels': 2, 'year': '1954', 'owner': 'UC', 'mp_code': 'P', 'book_value': 106819}, {'location': 'IRVINE', 'building': '9207', 'building_name': 'ALUMNI HOUSE', 'address': 'CORE CAMPUS', 'city_name': 'IRVINE', 'county': 'ORANGE', 'category': 'GENERAL', 'osfg': 4027, 'bldg_asf': 2549, 'levels': 1, 'year': '1984', 'owner': 'UC', 'mp_code': 'P', 'book_value': 41981}, {'location': 'FRANCISC SOAN', 'building': '2032', 'building_name': 'ALUMNI HOUSE', 'address': 'PARNASS US AVE', 'city_name': 'FRANCISC OSAN', 'county': 'FRANCISC OSAN', 'category': 'HEALTH SCIENCE', 'osfg': 7217, 'bldg_asf': 5079, 'levels': 3, 'year': '1915', 'owner': 'UP', 'mp_code': 'P', 'book_value': 135923}]"
CAMPBELL,"[{'location': 'BERKELEY', 'building': '1027', 'building_name': 'CAMPBELL', 'address': 'CORE C28A5M0PUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 83373, 'bldg_asf': 50745, 'levels': 7, 'year': '2014', 'owner': 'UC', 'mp_code': 'P', 'book_value': 75256344.0}, {'location': 'LOS ANGELES', 'building': '4294', 'building_name': 'CAMPBELL', 'address': 'PORTOLA P11L8A1Z1A', 'city_name': 'LOS ANGELES', 'county': 'LOS ANGELES', 'category': 'GENERAL', 'osfg': 55401, 'bldg_asf': 32619, 'levels': 5, 'year': '1954', 'owner': 'UC', 'mp_code': 'P', 'book_value': 6247192.0}]"
HERTZ,"[{'location': 'BERKELEY', 'building': '1423', 'building_name': 'HERTZ', 'address': 'CORE CAMPUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 31362, 'bldg_asf': 13518, 'levels': 4, 'year': '1958', 'owner': 'UC', 'mp_code': 'P', 'book_value': 3491181.0}, {'location': 'DAVIS', 'building': '9454', 'building_name': 'HERTZ', 'address': '7000 EAST 2A4V0E0 LAKE', 'city_name': 'LIVERMOR E', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 10904, 'bldg_asf': 8183, 'levels': 1, 'year': '1976', 'owner': 'UC', 'mp_code': 'P', 'book_value': 0}]"
SOUTH,"[{'location': 'BERKELEY', 'building': '1484', 'building_name': 'SOUTH', 'address': 'CORE CAMPUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 30401, 'bldg_asf': 15473, 'levels': 4, 'year': '1873', 'owner': 'UC', 'mp_code': 'P', 'book_value': 1426174.0}, {'location': 'DAVIS', 'building': '3275', 'building_name': 'SOUTH', 'address': 'CORE CAMPUS', 'city_name': 'DAVIS (YOLO)', 'county': 'YOLO', 'category': 'GENERAL', 'osfg': 26594, 'bldg_asf': 15323, 'levels': 4, 'year': '1912', 'owner': 'UC', 'mp_code': 'P', 'book_value': 3585399.0}]"
SPROUL,"[{'location': 'BERKELEY', 'building': '1210', 'building_name': 'SPROUL', 'address': 'CORE 2C6A0M0PUS', 'city_name': 'BERKELEY', 'county': 'ALAMEDA', 'category': 'GENERAL', 'osfg': 111197, 'bldg_asf': 66680, 'levels': 6, 'year': '1941', 'owner': 'UC', 'mp_code': 'P', 'book_value': 4278819.0}, {'location': 'DAVIS', 'building': '3815', 'building_name': 'SPROUL', 'address': 'CORE CAMPUS', 'city_name': 'DAVIS (YOLO)', 'county': 'YOLO', 'category': 'GENERAL', 'osfg': 53874, 'bldg_asf': 27913, 'levels': 10, 'year': '1963', 'owner': 'UC', 'mp_code': 'P', 'book_value': 1620880.0}, {'location': 'RIVERSIDE', 'building': 'P5523', 'building_name': 'SPROUL', 'address': 'CORE CAMPUS', 'city_name': 'RIVERSIDE', 'county': 'RIVERSIDE', 'category': 'GENERAL', 'osfg': 80988, 'bldg_asf': 45532, 'levels': 5, 'year': '1965', 'owner': 'UC', 'mp_code': 'P', 'book_value': 1296147.0}]"


<br><br>

---
### Question 1c

Now find examples of many matches in the opposite direction. For each distinct `real_estate_metadata.building_name` value, find the ones that have multiple matches in `buildings_site_mapping.building`, and for each return a `json_agg` of the multiple values for `buildings_site_mapping`. Your output should contain the building name and the `json_agg` in that order. **Order your final result by building name ascending.**

**Hint:** You should use a CTE to find the distinct building names of `real_estate_metadata` before applying necessary table joins.

In [12]:
%%sql --save query_1c result_1c <<
with tbl as (
select distinct building_name 
from real_estate_metadata 
)
select r.building_name, json_agg(b)
from buildings_site_mapping as b
join tbl as r 
on b.building = r.building_name
group by r.building_name 
having count(*) > 1
order by r.building_name

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

4 rows affected.

In [13]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_1c = %sqlcmd snippets query_1c
grading_util.save_results("result_1c", query_1c, result_1c)
result_1c

building_name,json_agg
EDWARDS FLD,"[{'site': 'Edwards Stadium East', 'building': 'EDWARDS FLD'}, {'site': 'Edwards Stadium West', 'building': 'EDWARDS FLD'}]"
FAC CLUB,"[{'site': ""Men's Faculty Club"", 'building': 'FAC CLUB'}, {'site': 'Womens Faculty Club', 'building': 'FAC CLUB'}]"
HAAS STU BLD,"[{'site': 'Haas School Student Services (Water)', 'building': 'HAAS STU BLD'}, {'site': 'Haas School of Business', 'building': 'HAAS STU BLD'}]"
SIMON,"[{'site': 'Simpson Center', 'building': 'SIMON'}, {'site': 'Boalt and Simon Hall', 'building': 'SIMON'}]"


<!-- END QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 2: Looking for Outliers in the Readings
Physical sensors, such as the ones responsible for generating this data, are notorious for occasionally producing crazy outliers. In this section, we will undergo some data cleaning to address these outliers.

The readings from all different types of sensors are mixed together within this `data` table. This assortment of mixed readings will require some extra work to identify the outliers. Let's get started.

### Question 2a: Outlier Detection

Let's start with finding the outlying values *for each sensor id*. We'll define an outlier as an observation that is more than **3 Hampel X84 intervals** away from the median. 

Create a view `labeled_data` that contains all of the columns in `data` and adds three additional columns on the far right ***for each sensor id***:
  - `median` containing the median using `percentile_disc`
  - `mad` containing the Median Absolute Deviation (MAD),
  - `outlier` that contains `true` for the outlier readings and `false` for the rest. **Also,** for data points where the `mad` is 0, set this to `false`.

In [15]:
%%sql
create or replace view tbl1 as 
    select id, 
    percentile_disc(0.5) within group (order by value) as median
    from data
    group by id

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

++
||
++
++

In [16]:
%%sql
create or replace view tbl2 as 
select data.time, data.id, data.value, tbl1.median, abs(tbl1.median - value) as stddev
    from data 
    inner join tbl1 on data.id = tbl1.id

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

++
||
++
++

In [17]:
%%sql
create or replace view tbl3 as 
select id, percentile_disc(0.5) within group (order by stddev) as MAD
    from tbl2 
    group by id

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

++
||
++
++

In [18]:
%%sql
create or replace view tbl4 as
select t2.time, t2.id, t2.value, t2.median, t2.stddev, t3.MAD
    from tbl2 as t2
    inner join tbl3 as t3
    on t2.id = t3.id

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

++
||
++
++

In [19]:
%%sql --save query_2a result_2a <<
CREATE OR REPLACE VIEW labeled_data AS
select time, id, value, median, MAD, 
case when stddev between mad * 1.4826 * -3 and mad * 1.4826 * 3 then false
when mad = 0 then false
else true end as outlier
from tbl4
;
SELECT * FROM labeled_data WHERE outlier ORDER BY id, time LIMIT 100;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

100 rows affected.

In [20]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_2a = %sqlcmd snippets query_2a
grading_util.save_results("result_2a", query_2a, result_2a)
result_2a

time,id,value,median,mad,outlier
2018-06-09 08:45:00+00:00,a47e9bd4-ab61-56d5-9e0e-2dce29a46dd3,98.7,37.2,12.5,True
2018-06-07 00:00:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,812.6,727.5,16.200000000000045,True
2018-06-07 18:00:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,807.3,727.5,16.200000000000045,True
2018-06-07 18:15:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,800.0,727.5,16.200000000000045,True
2018-06-07 18:30:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,819.5,727.5,16.200000000000045,True
2018-06-07 18:45:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,808.7,727.5,16.200000000000045,True
2018-06-07 19:15:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,811.1,727.5,16.200000000000045,True
2018-06-07 19:30:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,804.2,727.5,16.200000000000045,True
2018-06-07 19:45:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,810.9,727.5,16.200000000000045,True
2018-06-07 20:00:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,801.7,727.5,16.200000000000045,True


<br><br>

---

### Question 2b: Outlier Handling (Winsorization)

In this step, define a view `cleaned_data` containing all the columns of `labeled_data` and one additional column on the far right called `clean_value`. This column will contain a copy of `labeled_data.value` if that value is not an outlier. For outliers, it should contain the value Winsorized to the nearest outlier boundary value (3 Hampel X84 intervals from the median). If the MAD is 0, then the cleaned value should be unchanged (i.e., the same as the original value).

To reiterate, we consider a value to be an outlier if it is more than (strictly greater than or strictly less than) 3 Hampel X84 intervals from the median. 

In [22]:
%%sql
create or replace view tbl_2b as
select ld.time, ld.id, ld.value, ld.median, ld.mad, ld.outlier, h.hampel
from labeled_data as ld
inner join (
    select temp.id, temp.MAD * 3 * 1.4826 as hampel 
    from (
    select id, 
    percentile_disc(0.5) within group (order by stddev) as MAD
    from tbl2 
    group by id) as temp
) as h
on ld.id = h.id

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

++
||
++
++

In [23]:
%%sql --save query_2b result_2b <<
CREATE OR REPLACE VIEW cleaned_data AS
select *, 
    case 
    when mad = 0 then value
    when outlier = False then value
    when (value >= median) then median + hampel
    else
    median - hampel
end as clean_value
from tbl_2b
;
SELECT time, id, value, median, mad, outlier, clean_value
FROM cleaned_data WHERE outlier ORDER BY id, time LIMIT 100;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

100 rows affected.

In [24]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_2b = %sqlcmd snippets query_2b
grading_util.save_results("result_2b", query_2b, result_2b)
result_2b

time,id,value,median,mad,outlier,clean_value
2018-06-09 08:45:00+00:00,a47e9bd4-ab61-56d5-9e0e-2dce29a46dd3,98.7,37.2,12.5,True,92.7975
2018-06-07 00:00:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,812.6,727.5,16.200000000000045,True,799.5543600000002
2018-06-07 18:00:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,807.3,727.5,16.200000000000045,True,799.5543600000002
2018-06-07 18:15:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,800.0,727.5,16.200000000000045,True,799.5543600000002
2018-06-07 18:30:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,819.5,727.5,16.200000000000045,True,799.5543600000002
2018-06-07 18:45:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,808.7,727.5,16.200000000000045,True,799.5543600000002
2018-06-07 19:15:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,811.1,727.5,16.200000000000045,True,799.5543600000002
2018-06-07 19:30:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,804.2,727.5,16.200000000000045,True,799.5543600000002
2018-06-07 19:45:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,810.9,727.5,16.200000000000045,True,799.5543600000002
2018-06-07 20:00:00+00:00,a481d4a8-44f6-5f63-b469-85571a018c6f,801.7,727.5,16.200000000000045,True,799.5543600000002


<!-- BEGIN QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 3: Entity Resolution





### Question 3b

Sometimes entity resolution is as simple as a text transformation. For example, how many unique `units` values are there, and how many would there be if we ignored case (upper vs. lower case)? Your output should be a table with one row and two columns; the first column should contain the number of unique `units` values, and the second column should contain the number of unique `units` values if we ignored case. The two columns can have arbitrary names—we will not be checking column names.


In [27]:
%%sql --save query_3b result_3b <<
select count(distinct units) as unique, count(distinct upper(units)) as unique2
from metadata

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

1 rows affected.

In [28]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_3b = %sqlcmd snippets query_3b
grading_util.save_results("result_3b", query_3b, result_3b)
result_3b

unique,unique2
34,29


<br><br>

---

### Question 3c

Arguably, we shouldn't care about these alternative unit labels, *as long as each sensor **class** uses a single value of `units` for all its sensor ids*. After all, maybe the capitalization means something to somebody!

Write a SQL query that returns a **single row with one column** of value `true` if the condition (in italics above) holds, or a single row with one column of value `false` otherwise. This column can have an arbitrary name—we will not be checking its name. Please do not hard code this query—we reserve the right to penalize your score if you do so.


In [30]:
%%sql --save query_3c result_3c <<
select 
  not exists (
    select class
    from (
      select class, count(distinct upper(units)) as unit_count
      from metadata
      group by class
      having count(distinct lower(units)) > 1
    ) as s
  ) as condition;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

1 rows affected.

In [31]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_3c = %sqlcmd snippets query_3c
grading_util.save_results("result_3c", query_3c, result_3c)
result_3c

condition
True


<!-- END QUESTION -->

<br><br>

---
### Question 3e

It turns out this `real_estate_metadata` table is the result of an [OCR scan](https://en.wikipedia.org/wiki/Optical_character_recognition). We will just focus on cleaning up the `location` column for the time being, and leave you to imagine the effort required to do a full cleanup of all columns.

We have preloaded Postgres' extension packages for "fuzzy" string matching (`fuzzystrmatch`) and trigrams (`pg_trgm`) for you so that you can take advantage of some convenient utility functions. You can use any of the string functions in those packages if you'd like ([as documented here for fuzzystrmatch](https://www.postgresql.org/docs/current/fuzzystrmatch.html) or [here for pg_trgm](https://www.postgresql.org/docs/current/pgtrgm.html)).

We also created a lookup table of standardized names, `uc_locations`.

Now, using any of the string functions you like (or none at all!), write a SQL query that returns the columns `(building_name, address, location, clean_location)` where `clean_location` contains the best match from `uc_locations.loc_name`. We recommend taking a look at `levenshtein`, `word_similarity`, `metaphone`, but feel free to explore the different functions available. You may find that you can't clean up everything with the string functions, so your view may have to include some specific logic for cases in the data that have to be handled "manually". You can choose to do this question in whatever manner you wish as long as your query does not use `CREATE TABLE`, `INSERT INTO`, or `UPDATE`.

In [34]:
%%sql --save query_3e result_3e <<
select 
r.building_name,
r.address,
r.location,
u.loc_name as clean_location
from 
real_estate_metadata r
left join uc_locations u 
on u.loc_name = (
select loc_name
from uc_locations
order by word_similarity(loc_name, r.location) desc, 
levenshtein(loc_name, r.location)
limit 1
);

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

5276 rows affected.

In [35]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_3e = %sqlcmd snippets query_3e
grading_util.save_results("result_3e", query_3e, result_3e)
result_3e

building_name,address,location,clean_location
CLEARY,2424 CHANNING,BERKELEY,BERKELEY
WALNUT19 25,1925 WALNUT,BERKELEY,BERKELEY
ERCHLDE DCNTR,2339 HASTE,BERKELEY,BERKELEY
MRI MODLR II,CORE 2C1A5M4P-6U0S,BERKELEY,BERKELEY
UNIV 2154- 60,UNIVERSIT 2Y55,BERKELEY,BERKELEY
HAVENS,PANORAMI C25 W5 AY,BERKELEY,BERKELEY
HAVENS GR,PANORAMI 2C9 W99A Y,BERKELEY,BERKELEY
BOAT HOUSE,GLASCOC K STREET,BERKELEY,BERKELEY
BOAT EBRIGHT,303 DERBY AVENUE,BERKELEY,BERKELEY
BOAT SHED,303 DERBY AVENUE,BERKELEY,BERKELEY


<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 4: Interpolating Missing Data
Real-world data, real-world problems. Our sensors should be reporting every 15 minutes, but you can be sure that we're missing some data. Here we will fix it. It's a bit more involved than what we looked at in class!

### Question 4a: Finding missing readings
In the `data` table, the `id` column identifies a unique sensor. Sensor readings should be recorded every 15 minutes from every sensor. Are we missing any readings, and if so which ones? We will focus on readings that are separated by at least 30 minutes or more; readings that are \[0-30) minutes apart are considered to be fine.

To answer this question you'll need to read up a bit on [SQL timestamps](https://www.postgresql.org/docs/current/datatype-datetime.html) and [Functions for manipulating datetime types](https://www.postgresql.org/docs/current/functions-datetime.html). Have a particular look at the following:
- The [date_trunc](https://www.postgresql.org/docs/current/functions-datetime.html#FUNCTIONS-DATETIME-TRUNC) function will quantize times to the nearest unit of your choosing. For example, to round the `time` field to the nearest minute you can say `date_trunc('minute', time)`. **You'll need to quantize to minutes right away before you worry about missing readings.**
- There are various ways to enter constant intervals of time as strings. For example, a 30 minute interval can be written as `interval '30 minutes'` or `'30 minutes'::interval`. See [date/time input](https://www.postgresql.org/docs/current/datatype-datetime.html#DATATYPE-INTERVAL-INPUT) for more info.
- You can do arithmetic on date/time types [as documented here](https://www.postgresql.org/docs/current/functions-datetime.html#FUNCTIONS-DATETIME-TRUNC). That will handle all the weird periodicities of clocks and calendars for you. Pay attention to the input and output types of these functions!
- Alternatively, the [EXTRACT](https://www.postgresql.org/docs/current/functions-datetime.html#FUNCTIONS-DATETIME-EXTRACT) function is sometimes handy. Note the special `EXTRACT(EPOCH FROM ...)` case. This converts a timestamp into an integer representing the number of seconds since midnight, 1/1/1970 (the dawn of [UNIX time](https://en.wikipedia.org/wiki/Unix_time)!)  You can do normal integer comparisons and arithmetic on the results.
- You will need to use the [lag](https://www.geeksforgeeks.org/postgresql-lag-function/) function as the window function.


Create a view called `gaps` that augments the `data` schema with three columns:
- `lagtime` is the quantized time of the previous reading for that sensor (relative to the current row for a particular row)
- `lagvalue` is the value of the previous reading for that sensor
- `timediff` is the difference in quantized time between this reading and the previous reading

The view should only contain rows where `timediff` is **greater than or equal to 30 minutes**.

In [37]:
%%sql --save query_4a result_4a <<
CREATE OR REPLACE VIEW gaps AS
WITH sensor_readings AS (
  SELECT 
    id,
    time,
    value,
    date_trunc('minute', lag(time) OVER (PARTITION BY id ORDER BY time)) AS lagtime,
    lag(value) OVER (PARTITION BY id ORDER BY time) AS lagvalue,
    EXTRACT(EPOCH FROM (time - lag(time) OVER (PARTITION BY id ORDER BY time))) / 60 AS timediff
  FROM 
    data
)
SELECT 
  id,
  time,
  value,
  lagtime,
  lagvalue,
  timediff
FROM 
  sensor_readings
WHERE 
  timediff >= 30;
SELECT * FROM gaps ORDER BY id, time LIMIT 10;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

10 rows affected.

In [38]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_4a = %sqlcmd snippets query_4a
grading_util.save_results("result_4a", query_4a, result_4a)
result_4a

id,time,value,lagtime,lagvalue,timediff
a3d47b1a-985e-5395-a6ee-719dad9b580f,2018-06-13 09:15:00+00:00,0.056,2018-06-13 08:45:00+00:00,0.056,30.0000000000000000
a3e8e405-0eed-59b6-8747-fe892a6f93de,2018-06-09 04:15:00+00:00,280.8,2018-06-09 03:45:00+00:00,281.6,30.0000000000000000
a46fb790-028c-5d17-a3e6-7d08daec0c03,2018-06-13 09:15:00+00:00,281.46,2018-06-13 08:45:00+00:00,281.39,30.0000000000000000
a470ac6d-d448-522b-9b74-8ed27f24de25,2018-06-09 04:15:00+00:00,0.004,2018-06-09 03:45:00+00:00,0.006,30.0000000000000000
a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 00:45:02+00:00,27.273,2018-06-07 00:15:00+00:00,30.0,30.0166666666666667
a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 18:45:07+00:00,25.0,2018-06-07 18:15:00+00:00,30.0,30.0833333333333333
a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 19:45:05+00:00,23.077,2018-06-07 19:15:00+00:00,20.0,30.0333333333333333
a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 20:30:04+00:00,23.077,2018-06-07 20:00:00+00:00,30.0,30.0166666666666667
a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 21:00:04+00:00,14.286,2018-06-07 20:30:00+00:00,23.077,30.0000000000000000
a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 22:15:05+00:00,30.0,2018-06-07 21:45:00+00:00,20.0,30.0500000000000000


### Question 4b: Creating tuples for the missing readings
Now we need to manufacture new tuples to fill in the gaps. For example, if you had a tuple from id `abc` timestamped at 1PM today and the next tuple in time from `abc` was timestamped at 1:45PM, you'll need to manufacture two new tuples with id `abc` and `NULL` values: one timestamped at 1:15PM and another timestamped at 1:30PM. We will worry about replacing the `NULL` values in the next step.





Create a view `complete` that contains the tuples from `data` as well as new tuples that fill in any gaps greater than or equal to 30 minutes. Each gap should be filled by adding tuples in increments of 15 minutes from the *start* of the gap, **with `NULL` as the value**. You probably want to use your `gaps` view as well as `generate_series` to do this!

**Hint:** the lower and upper bounds in generate_series (in pseudocode) should be `(lagtime + 15 minutes, time - 15 minutes)`.

In [42]:
%%sql --save query_4b result_4b <<
CREATE OR REPLACE VIEW complete AS
SELECT 
  d.id,
  d.time,
  d.value
FROM 
  data d

UNION ALL

SELECT 
  g.id,
  gs AS time,
  NULL AS value
FROM 
  gaps g,
  generate_series(g.lagtime + interval '15 minutes', g.time - interval '15 minutes', interval '15 minutes') gs
ORDER BY 
  id, 
  time;

SELECT * FROM complete ORDER BY id, time LIMIT 100;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

100 rows affected.

In [43]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_4b = %sqlcmd snippets query_4b
grading_util.save_results("result_4b", query_4b, result_4b)
result_4b

id,time,value
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:00:09+00:00,65085.99
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:15:09+00:00,65086.16
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:30:09+00:00,65086.35
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:45:09+00:00,65086.52
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 01:00:09+00:00,65086.71
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 01:15:09+00:00,65086.89
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 01:30:09+00:00,65087.07
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 01:45:09+00:00,65087.25
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 02:00:09+00:00,65087.43
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 02:15:09+00:00,65087.62


### Question 4c: Linear Interpolation



Create a view `likely_data` that contains all the tuples from `complete`, with an additional column called `interpolated` that contains a copy of `value` if it is non-NULL, otherwise an interpolated value based on linear interpolation **per sensor id over time**. Please **retain all additional columns created from forward and backward passes**. (There should be 10 columns in the view—the order of the columns does not matter.) The three cells below correspond to the forward, backward, and final passes from lecture.

In [45]:
%%sql
CREATE OR REPLACE VIEW forward AS
SELECT *,
    SUM(CASE WHEN value IS NULL THEN 0 ELSE 1 END)
        OVER (ORDER BY id, time) AS run,
    coalesce_agg(value) OVER (ORDER BY id, time) AS run_start,
    CASE WHEN value IS NULL THEN lead(value, 1) OVER (ORDER BY id, time) ELSE NULL
    END as next
FROM complete_provided;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

++
||
++
++

In [46]:
%%sql
CREATE OR REPLACE VIEW backward AS
SELECT *, CASE WHEN value is not null then value else coalesce_agg(next) over (partition by run order by id, time desc) end as run_end,
count(*) over (partition by run) as run_size, count(*) over (partition by run order by id,time) as run_rank 
from forward;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

++
||
++
++

In [47]:
%%sql --save query_4c result_4c <<
CREATE OR REPLACE VIEW likely_data AS
select *, run_start + (run_rank -1) * (run_end - run_start)/run_size as interpolated
from backward
;
SELECT * FROM likely_data WHERE run_size > 2 ORDER BY id, time LIMIT 100;

Running query in 'postgresql://jovyan@127.0.0.1:5432/ucb_buildings'

100 rows affected.

In [48]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_4c = %sqlcmd snippets query_4c
grading_util.save_results("result_4c", query_4c, result_4c)
result_4c

id,time,value,run,run_start,next,run_end,run_size,run_rank,interpolated
a49cd747-43c3-536d-b3ee-ec9c087b9317,2018-06-08 10:00:02+00:00,1.7230274441963545,10646,1.7230274441963545,None,1.7230274441963545,3,1,1.7230274441963545
a49cd747-43c3-536d-b3ee-ec9c087b9317,2018-06-08 10:15:00+00:00,None,10646,1.7230274441963545,None,5.372979444921953,3,2,2.939678111104887
a49cd747-43c3-536d-b3ee-ec9c087b9317,2018-06-08 10:30:00+00:00,None,10646,1.7230274441963545,5.372979444921953,5.372979444921953,3,3,4.156328778013419
a49cd747-43c3-536d-b3ee-ec9c087b9317,2018-06-08 19:45:03+00:00,0.6144687255293491,10683,0.6144687255293491,None,0.6144687255293491,3,1,0.6144687255293491
a49cd747-43c3-536d-b3ee-ec9c087b9317,2018-06-08 20:00:00+00:00,None,10683,0.6144687255293491,None,8.10343967255239,3,2,3.1107923745370294
a49cd747-43c3-536d-b3ee-ec9c087b9317,2018-06-08 20:15:00+00:00,None,10683,0.6144687255293491,8.10343967255239,8.10343967255239,3,3,5.60711602354471
a49cd747-43c3-536d-b3ee-ec9c087b9317,2018-06-09 01:00:03+00:00,2.196116776472188,10702,2.196116776472188,None,2.196116776472188,3,1,2.196116776472188
a49cd747-43c3-536d-b3ee-ec9c087b9317,2018-06-09 01:15:00+00:00,None,10702,2.196116776472188,None,1.4019035586474615,3,2,1.931379037197279
a49cd747-43c3-536d-b3ee-ec9c087b9317,2018-06-09 01:30:00+00:00,None,10702,2.196116776472188,1.4019035586474615,1.4019035586474615,3,3,1.6666412979223701
a49cd747-43c3-536d-b3ee-ec9c087b9317,2018-06-10 05:00:02+00:00,4.692789500399002,10813,4.692789500399002,None,4.692789500399002,3,1,4.692789500399002


<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />